# DO NOT USE
We only get 5 API calls a minute.

In [ ]:
import pandas as pd
import get_data
import save_data_to_csv as save
import time
import numpy as np


table=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
sp_ticks = table[0]['Symbol']
counter = 0
# for sym in sp_ticks:
#     try:
#         save.save_dataset(sym, timeframe='daily')
#         counter += 1
#         print(counter)
#         time.sleep(12.1)
#     except:
#         continue
# at 210 (so around 291 total???)

In [ ]:
# from here on out sp_ticks is unnecessary
sp_adj_closes = {}
for n in range(250):
    try:
        sp_adj_closes[sp_ticks[n]] = get_data.get_data(sp_ticks[n]).sort_values('date')['5. adjusted close']
    except:
        pass
sp_adj_closes['MMM']

In [ ]:
trend_days = 30    # days
drop_days = 1     # days
drop_percent = .1  # percent
time_after = 60   # days

buy_date = '2018-01-04'
sell_date = '2020-01-31'

In [ ]:
sp_meta = {} # AAPL: (stock price at start date, stock price at end date, how much hold strat would've made, index in Series of start date, index in Series of end date)
for tick in sp_adj_closes:
    try:
        price_data = sp_adj_closes[tick]
        
        buy_price = price_data.loc[buy_date]
        sell_price = price_data.loc[sell_date]
        hold_strat_earnings = sell_price - buy_price
        
        start_ind = price_data.index.get_loc(buy_date)
        end_ind = price_data.index.get_loc(sell_date)
        sp_meta[tick] = (buy_price, sell_price, hold_strat_earnings, start_ind, end_ind)
        
        print("If you had bought", tick, "on", buy_date, "and sold it on", sell_date, "you would have made", hold_strat_earnings)
    except:
        pass

In [ ]:
def sim(trend_days, drop_days, drop_percent, time_after):
    wallet = 0
    money_invested = 0
    for tick in sp_adj_closes:
        try:
            buy_price, sell_price, hold_strat_earnings, start_ind, end_ind = sp_meta[tick]
            price_data = sp_adj_closes[tick]
            gains_counter = 0
            drop_counter = 0 
            for n in range(start_ind + trend_days, end_ind - time_after):
                today_price = price_data[n]
                drop_day_price = price_data[n+drop_days]
                recovery_price = price_data[n+time_after]
                stays_afloat = True
                if price_data[n - trend_days] > today_price: # if the price didn't decrease
                    stays_afloat = False
#                 for j in range(n - trend_days, n):
#                     if price_data[j + drop_days] < (1 - drop_percent) * price_data[j]:
#                         stays_afloat = False
#                         break
                if stays_afloat:
                    if drop_day_price <= (1 - drop_percent) * today_price:
                        drop_counter += 1
                        money_invested += drop_day_price
                        wallet -= drop_day_price
                        wallet += recovery_price
                        #if price_data[n+time_after] >= price_data[n]:
                        if recovery_price >= drop_day_price:
                            gains_counter += 1                        
        except:
            pass
    return(wallet, money_invested)

In [ ]:
def optimize():
    count = 0
    data = []
    for trend in range(5, 100, 5):
        for drop in range(1, 10):
            for drop_p in range(5, 20):
                for rec in range(10, 200):
                    wallet, money_invested = sim(trend, drop, drop_p / 100, rec)
                    data.append(np.array([trend, drop, drop_p / 100, rec, wallet, money_invested]))
                    count += 1
                    #if count % 10 == 0:
                    print(count)
                    print(wallet, money_invested)
    return np.array(data)

In [ ]:
optimize()